In [1]:
import numpy as np
from astropy.table import Table
import astropy.io.fits as pyfits
from astropy import units as u
from astropy.coordinates import SkyCoord
import copy
import sys
import yaml
import h5py as h

"""
The expected structure is for flat fits files with the photometry to be in the form of 2-D arrays, as are produced by Erin's model fitting code. 
"""
        
print("here")

here


In [2]:
print("here")
def read_cat(in_cat):
    try:
        t = Table.read(in_cat)
    except:
        print('Error: catalogue ',in_cat,' not found or not a fits table!')
        data = h.File(catname,'r')
        t = np.array(f['catalog/gold'])#[star_mask] 
        #sys.exit()
    return t


def match_cats(ra1, dec1, ra2, dec2):
    # ra1, dec1 are the deep cat.
    c = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)  
    catalog = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)  
    idx, d2d, d3d = c.match_to_catalog_sky(catalog)
    return idx

    
def deredden(t, config):
    try:
        # get column names and extinction coeffs from config
        fcol = config['flux_col']
        efcol = config['flux_err_col']
        mcol = config['mag_col']
        coeffs = config['coeffs']

        # build extinction array
        extinct = np.array(t['EBV_SFD98'].quantity).repeat(len(coeffs)).reshape((-1,len(coeffs))) * coeffs

        # apply corrections
        t[fcol+'_dered'] = t[fcol] * 10.**(0.4*extinct)
        t[efcol+'_dered'] = t[efcol] * 10.**(0.4*extinct)
        t[mcol+'_dered'] = t[mcol] - extinct
        t[mcol+'_err_dered'] = 2.5*np.log10(1.+(t[efcol]/t[fcol]))

        # clean up inf and nan to flag values
        t[fcol+'_dered'][np.isnan(t[fcol+'_dered'])] = -9.999e9
        t[mcol+'_dered'][np.isnan(t[mcol+'_dered'])] = -9.999e9
        t[efcol+'_dered'][np.isnan(t[efcol+'_dered'])] = 9.999e9
        t[mcol+'_err_dered'][np.isnan(t[mcol+'_err_dered'])] = 9.999e9

        t[fcol+'_dered'][np.isinf(t[fcol+'_dered'])] = -9.999e9
        t[mcol+'_dered'][np.isinf(t[mcol+'_dered'])] = -9.999e9
        t[efcol+'_dered'][np.isinf(t[efcol+'_dered'])] = 9.999e9
        t[mcol+'_err_dered'][np.isinf(t[mcol+'_err_dered'])] = 9.999e9

    except:
        print('Error: catalogue columns missing or have unexpected names')
        sys.exit()

    # write out
    print(config['deep_cat'].split('.')[0]+'_extcorr.fits')
    t.write(config['deep_cat'].split('.')[0]+'_extcorr.fits', overwrite=True)
    
        

here


In [ ]:

if __name__ == '__main__':

    # read in config
    if len(sys.argv) < 2:
        print('Use: python DES_deep_ext_corr.py <yaml config file>')
        sys.exit()
    else:
        #config = yaml.load(open(sys.argv[1]))
        config = yaml.load(open('dered.yaml'))

    deep = read_cat(config['deep_cat'])
    gold = read_cat(config['gold_cat'])
    
    print(deep)

    print(gold)
    # match cats
    idx = match_cats(deep[config['coords'][0]], deep[config['coords'][1]], gold[config['coords'][2]], gold[config['coords'][3]])

    # add extinction column to deep cat
    deep[config['Ext_col']] = gold[config['Ext_col']][idx]

    # free this up if we can
    del(gold)
    
    deredden(deep, config)


    

"""
Alt style if the columns have names:


# perform corrections and write table
try:
    for j,filt in enumerate(bands):
        fcol = 'bdf_flux_{}'.format(filt)
        efcol = 'bdf_flux_err_'.format(filt)
        mcol = 'bdf_mag_{}'.format(filt)
        emcol = 'bdf_mag_err_{}'.format(filt)
        #
        t[mcol+'_dered'] = t[mcol] - coeffs[j]*t['EBV_SFD98']
        t[emcol+'_dered'] = t[emcol]
        t[fcol+'_dered'] = t[fcol] * 10.**(0.4*coeffs[j]*t['EBV_SFD98'])
        t[efcol+'_dered'] = t[efcol] * 10.**(0.4*coeffs[j]*t['EBV_SFD98'])
    t.write(in_cat.split('.')[0]+'_extcorr.fits')
except:
    print('Error: catalogue columns missing or have unexpected names')
    sys.exit()

"""


   id           ra         ...             bdf_flux_err [9]            
------- ------------------ ... ----------------------------------------
      1 149.49435177756047 ...  21.82269586009592 .. 31.714303367568057
      2 149.49490152216228 ...   42.99559508131026 .. 73.57418236093281
      3 149.49379397755666 ...  8.058362738815294 .. 10.324133723812867
      5 149.49149526579927 ...  12.836735187239222 .. 16.62117511084364
      6 149.49141694256778 ... 12.785379886081964 .. 16.452129650402775
      8 149.49187176561608 ... 13.438655234936544 .. 20.592997235816824
      9 149.48553342140582 ... 14.281923752591993 .. 19.235384953689422
     11  149.4904211930061 ... 11.264357798664458 .. 17.622102567051034
     12  149.4871599313461 ...   7.506435951234584 .. 8.821826317475189
     13  149.4913734409627 ...  14.26234850352879 .. 24.849045917848148
    ...                ... ...                                      ...
1177219 150.73542051128635 ...             9999000000.0 .. 99990